In [1]:
import pymysql


def connect_db():
    
    return pymysql.connect(
        host='localhost',
        user='root',
        password='mathi',
        database='payroll_management'
    )


def sign_up():
    
    username = input("Enter username: ") 
    password = input("Enter password: ") 
    role = input("Enter role (admin/employee): ").lower()
    connection = connect_db()
    cursor = connection.cursor()

    if role == "admin":
       cursor.execute("SELECT * FROM users WHERE username = %s", (username,))
       
       if cursor.fetchone():
           print("Username already exists in admin database.")
           return
        
       cursor.execute("INSERT INTO users (username, password, role) VALUES (%s, %s, %s)", (username, password, role))
       
    elif role == "employee":
        cursor.execute("SELECT * FROM users WHERE username = %s", (username,))
        
        if cursor.fetchone():
            print("Username already exists.")
            return
        
        cursor.execute("INSERT INTO users (username, password, role) VALUES (%s, %s, %s)", (username, password, role))
        
    else:
        print("Invalid role. Please choose 'admin' or 'employee'.")
        return

    connection.commit()
    
    print("Sign-up successful!")
    
    connection.close()


def login():
    
    username = input("Enter username: ") 
    password = input("Enter password: ")
    role = input("Enter role (admin/employee): ").lower()
    connection = connect_db()
    cursor = connection.cursor()

    if role == "admin": 
        cursor.execute("SELECT * FROM users WHERE username = %s AND password = %s AND role = 'admin'", (username, password))
        
    elif role == "employee": 
        cursor.execute("SELECT * FROM users WHERE username = %s AND password = %s AND role = 'employee'", (username, password))
        
    else:
        print("Invalid role. Please choose 'admin' or 'employee'.")
        return False
 
    user = cursor.fetchone() 
 
    if user: 
        print(f"Login successful! Role: {role.capitalize()}")
        
        if role == 'admin':
            admin_operations()
            
        elif role == 'employee':
            employee_operations()
        return True
    
    else: 
        print("Invalid username or password.")
        return False
    
    connection.close()


def admin_operations():
    
    while True:
        print("\nAdmin Menu")
        print("1. Add employee")
        print("2. View employees")
        print("3. See requests")
        print("4. Calculate salary of employee")
        print("5. view service period")
        print("6. Delete employee")
        print("7. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            add_employee()
        elif choice == '2':
            view_employees()
        elif choice == '3':
            see_requests()
        elif choice == '4':
            calculate_salary()
        elif choice == '5':
            view_service_period()
        elif choice == '6':
            delete_employee()
        elif choice == '7':
            break
        else:
            print("Invalid choice")


def employee_operations():
    
    while True:
        print("\nEmployee Menu")
        print("1. Send request")
        print("2. Withdraw salary")
        print("3. See my service period")
        print("4. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            send_request()
        elif choice == '2':
            withdraw_salary()
        elif choice == '3':
            see_service_period()
        elif choice == '4':
            break
        else:
            print("Invalid choice")


def add_employee():
    
    username = input("Enter employee username: ")
    password = input("Enter a temporary password for the employee: ")  
    salary = float(input("Enter  basic salary: "))
    service_period = int(input("Enter service period (in months): "))

    if not password:
        
        print("Error: Password cannot be empty.")
        return

    connection = connect_db()
    cursor = connection.cursor()

   
    cursor.execute("SELECT * FROM users WHERE username = %s", (username))
    user_exists = cursor.fetchone()

    if user_exists:
        print(f"User with username '{username}' already exists in the 'users' table.")
        
    else:
        try:
            
            cursor.execute("INSERT INTO users (username, password, role) VALUES (%s, %s, 'employee')", (username, password))
            connection.commit()

            cursor.execute("INSERT INTO employees (username, basic_salary, service_period) VALUES (%s, %s, %s)", (username, salary, service_period))
            connection.commit()

            print("Employee added successfully.")
            
        except pymysql.MySQLError as e:
            print(f"Error: {e}")
            connection.rollback()

    connection.close()


def view_employees():
    
    connection = connect_db()
    cursor = connection.cursor()

    cursor.execute("SELECT * FROM employees")
    employees = cursor.fetchall()
    
    for emp in employees:
        print(emp)
        print("-------------------------------------")
        
    connection.close()


def see_requests():
    
    connection = connect_db()
    cursor = connection.cursor()

    cursor.execute("SELECT * FROM requests")
    requests = cursor.fetchall()
    
    for req in requests:
        print(req)

    connection.close()


def check_and_add_column(cursor):
    
    cursor.execute("SHOW COLUMNS FROM employees LIKE 'total_salary'")
    result = cursor.fetchone()
    
   
    if result is None:
        cursor.execute("ALTER TABLE employees ADD total_salary FLOAT")
        print("Added 'total_salary' column to the 'employees' table.")


def calculate_salary():
    
    username = input("Enter employee username: ")

    connection = connect_db()
    cursor = connection.cursor()

    
    cursor.execute("SELECT basic_salary FROM employees WHERE username=%s", (username,))
    employee = cursor.fetchone()

    if employee:
        
        basic_salary = employee[0] 
        print(f"Basic Salary: {basic_salary}")
        
        
        overtime_hours = int(input("Enter overtime hours worked: "))
        overtime_rate = float(input("Enter overtime rate: "))
        
      
        average_overtime_hrs_per_month = 40
        hourly_rate = basic_salary / average_overtime_hrs_per_month
        
        
        overtime_pay = overtime_hours * overtime_rate * hourly_rate
        
        
        total_salary = basic_salary + overtime_pay
        
        print(f"Total Salary for {username}: {total_salary}")
        
        
        check_and_add_column(cursor)

       
        cursor.execute("UPDATE employees SET total_salary = %s WHERE username = %s", (total_salary, username))
        connection.commit()

        print(f"Total salary for {username} updated successfully in the database.")

    else:
        print("Employee not found")

    connection.close()


def view_service_period():
    
    username = input("Enter employee username: ")

    connection = connect_db()
    cursor = connection.cursor()

    cursor.execute("SELECT service_period FROM employees WHERE username=%s", (username,))
    employee = cursor.fetchone()

    if employee:
        print(f"Service period for {username} is {employee[0]} months")
    else:
        print("Employee not found")

    connection.close()


def delete_employee():
    
    username = input("Enter employee username to delete: ")

    connection = connect_db()
    cursor = connection.cursor()

    cursor.execute("DELETE FROM employees WHERE username=%s", (username,))
    connection.commit()

    if cursor.rowcount > 0:
        print("Employee deleted successfully")
    else:
        print("Employee not found")

    connection.close()


def send_request():
    
    username = input("Enter your username: ")
    request = input("Enter your request: ")

    connection = connect_db()
    cursor = connection.cursor()

    cursor.execute("INSERT INTO requests (username, request) VALUES (%s, %s)", (username, request))
    connection.commit()
    print("Request sent successfully")

    connection.close()


def withdraw_salary():
    
    username = input("Enter your username: ")

    connection = connect_db()
    cursor = connection.cursor()

    cursor.execute("SELECT total_salary FROM employees WHERE username=%s", (username,))
    employee = cursor.fetchone()

    if employee:
        
        total_salary = employee[0]
        print(f"Your current total salary is: {total_salary}")
        
        amount_withdraw = float(input("Enter the amount you wish to withdraw: "))
        
        if amount_withdraw > total_salary:
            print("Error: Cannot withdraw more than your total salary.")
            
        elif amount_withdraw <= 0:
            print("Error: Withdraw amount must be positive.")
            
        else:
            new_salary = total_salary - amount_withdraw
            print(f"Salary withdrawn: {amount_withdraw}. Remaining balance: {new_salary}")
            
            cursor.execute("UPDATE employees SET total_salary = %s WHERE username = %s", (new_salary, username))
            connection.commit()

            print("Total salary updated successfully in the database.")
    else:
        print("Employee not found.")

    connection.close()


def see_service_period():
    
    username = input("Enter your username: ")

    connection = connect_db()
    cursor = connection.cursor()

    cursor.execute("SELECT service_period FROM employees WHERE username=%s", (username,))
    employee = cursor.fetchone()

    if employee:
        print(f"Your service period is {employee[0]} months")
    else:
        print("Employee not found")

    connection.close()


def main():
    
    while True:
        print("\nPayroll Management System")
        print("1. Sign up")
        print("2. Login")
        print("3. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            sign_up()
        elif choice == '2':
            login()
        elif choice == '3':
            break
        else:
            print("Invalid choice")
            

if __name__ == "__main__":
    main()
    
    connection.close()



Payroll Management System
1. Sign up
2. Login
3. Exit
Sign-up successful!

Payroll Management System
1. Sign up
2. Login
3. Exit
Login successful! Role: Admin

Admin Menu
1. Add employee
2. View employees
3. See requests
4. Calculate salary of employee
5. view service period
6. Delete employee
7. Exit
Employee added successfully.

Admin Menu
1. Add employee
2. View employees
3. See requests
4. Calculate salary of employee
5. view service period
6. Delete employee
7. Exit
(3, 'Vidhya', 12000.0, 30, None)
-------------------------------------
(4, 'Esakkiammal', 12000.0, 36, None)
-------------------------------------
(5, 'Rina', 10000.0, 24, None)
-------------------------------------
(7, 'Harish', 11000.0, 24, None)
-------------------------------------
(19, 'ammu', 22000.0, 24, None)
-------------------------------------

Admin Menu
1. Add employee
2. View employees
3. See requests
4. Calculate salary of employee
5. view service period
6. Delete employee
7. Exit
(1, 'anu', '1')
(2, 'a